In [1]:
import pandas as pd
from datetime import datetime

path = "machine_learning_docs/melb_data.csv"

collisions = pd.read_csv(r"C:\Users\honzi\OneDrive\Desktop\py_osobni\machine_learning_docs\Traffic_Crashes_-_Crashes.csv")

print(collisions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832299 entries, 0 to 832298
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                832299 non-null  object 
 1   CRASH_DATE_EST_I               62073 non-null   object 
 2   CRASH_DATE                     832299 non-null  object 
 3   POSTED_SPEED_LIMIT             832299 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         832299 non-null  object 
 5   DEVICE_CONDITION               832299 non-null  object 
 6   WEATHER_CONDITION              832299 non-null  object 
 7   LIGHTING_CONDITION             832299 non-null  object 
 8   FIRST_CRASH_TYPE               832299 non-null  object 
 9   TRAFFICWAY_TYPE                832299 non-null  object 
 10  LANE_CNT                       199011 non-null  float64
 11  ALIGNMENT                      832299 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [2]:
print(collisions.columns)
print(collisions.isna().sum())

Index(['CRASH_RECORD_ID', 'CRASH_DATE_EST_I', 'CRASH_DATE',
       'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND',
       'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LATITUDE', 

In [4]:
#Check if there are no duplicate crash ID´s"
collisions["CRASH_RECORD_ID"].is_unique

#Simplify dates
collisions["CRASH_DATE"] = pd.to_datetime(collisions["CRASH_DATE"])
collisions["CRASH_DATE"] = collisions["CRASH_DATE"].dt.date

collisions["DATE_POLICE_NOTIFIED"] = pd.to_datetime(collisions["DATE_POLICE_NOTIFIED"])
collisions["DATE_POLICE_NOTIFIED"] = collisions["DATE_POLICE_NOTIFIED"].dt.date

# Define the cutoff date (start of 2017)
cutoff_date = datetime(2017, 1, 1).date()

# Filter the dataframe to keep only dates from 2017 and above
collisions = collisions[collisions["CRASH_DATE"] >= cutoff_date]


# Drop useless columns
collisions_cleaned = collisions.drop(["CRASH_RECORD_ID",
                                    "CRASH_DATE_EST_I",
                                    "LANE_CNT",
                                    "NOT_RIGHT_OF_WAY_I",
                                    "PHOTOS_TAKEN_I",
                                    "STATEMENTS_TAKEN_I",
                                    "DOORING_I",
                                    "WORK_ZONE_I",
                                    "WORK_ZONE_TYPE",
                                    "WORKERS_PRESENT_I",
                                    "BEAT_OF_OCCURRENCE",
                                    "INTERSECTION_RELATED_I"],axis = 1)
# Convert floats to integers
columns_to_convert = [
    "INJURIES_TOTAL",
    "INJURIES_FATAL",
    "INJURIES_INCAPACITATING",
    "INJURIES_NON_INCAPACITATING",
    "INJURIES_REPORTED_NOT_EVIDENT",
    "INJURIES_NO_INDICATION",
    "INJURIES_UNKNOWN"
]

collisions_cleaned = collisions_cleaned.dropna(subset = columns_to_convert)
collisions_cleaned[columns_to_convert] = collisions_cleaned[columns_to_convert].astype("int64")

# Drop rows when location is unknown
rows_to_drop = [
    "LONGITUDE",
    "LATITUDE",
    "LOCATION"
]
collisions_cleaned = collisions_cleaned.dropna(subset = rows_to_drop)

# Adjust total cost
to_replace_small_damage = "$0 - $500"
to_replace_big_damage = "$1500+"

collisions_cleaned['DAMAGE'] = collisions_cleaned['DAMAGE'].replace("$500 OR LESS", to_replace_small_damage)
collisions_cleaned['DAMAGE'] = collisions_cleaned['DAMAGE'].replace("OVER $1,500", to_replace_big_damage)

# Change wording
to_replace_desk_report = "DESK REPORT"

collisions_cleaned["REPORT_TYPE"] = collisions_cleaned["REPORT_TYPE"].replace("NOT ON SCENE (DESK REPORT)",to_replace_desk_report)

to_replace_crash_type = "INJURY / TOW DUE TO CRASH"

collisions_cleaned["CRASH_TYPE"] = collisions_cleaned["CRASH_TYPE"].replace("INJURY AND / OR TOW DUE TO CRASH", to_replace_crash_type)


# Fill missing values with N
collisions_cleaned['HIT_AND_RUN_I'].fillna('N', inplace=True)

# Assign days to numbers
day_mapping = {
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday"
}

collisions_cleaned["CRASH_DAY_OF_WEEK"] = collisions_cleaned["CRASH_DAY_OF_WEEK"].map(day_mapping)

# Assign months to numbers
month_mapping = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

collisions_cleaned["CRASH_MONTH"] = collisions_cleaned["CRASH_MONTH"].map(month_mapping)


In [5]:
pd.set_option('display.max_columns', None)
print(collisions_cleaned.head())

    CRASH_DATE  POSTED_SPEED_LIMIT TRAFFIC_CONTROL_DEVICE  \
1   2023-07-29                  30         TRAFFIC SIGNAL   
2   2023-08-18                  30            NO CONTROLS   
13  2023-07-29                  10            NO CONTROLS   
14  2023-07-29                  30         TRAFFIC SIGNAL   
20  2023-09-20                  15            NO CONTROLS   

        DEVICE_CONDITION WEATHER_CONDITION LIGHTING_CONDITION  \
1   FUNCTIONING PROPERLY             CLEAR           DAYLIGHT   
2            NO CONTROLS             CLEAR           DAYLIGHT   
13           NO CONTROLS           UNKNOWN            UNKNOWN   
14  FUNCTIONING PROPERLY             CLEAR           DARKNESS   
20           NO CONTROLS             CLEAR           DAYLIGHT   

                FIRST_CRASH_TYPE                  TRAFFICWAY_TYPE  \
1           PARKED MOTOR VEHICLE  DIVIDED - W/MEDIAN (NOT RAISED)   
2                   PEDALCYCLIST                      NOT DIVIDED   
13                         ANGLE   